# Using Batch API to call GLM models

**This tutorial is available in English and is attached below the Chinese explanation**

本章节将带领开发者使用 Batch API 来调用GLM系列模型。使用 Batch API将能以更低的价格调用模型，但是，需要等待一天的时间（batch API 是 非事实服务的）。
我们将按照[官方文档](https://open.bigmodel.cn/dev/howuse/batchapi)提供的方式进行调用。

This cookbook aims to help developers use the Batch API to call the GLM series of models. Using the Batch API, you can call the model at the price of the queue, but you need to wait for a day (the batch API is not a de facto service).
We will call it according to the method provided by [official documentation](https://open.bigmodel.cn/dev/howuse/batchapi).

# 1. Set up the API key

首先，设定好调用模型的API key
First, set the API key for calling the model

In [2]:
from zhipuai import ZhipuAI
import os

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

# 2. Upload data set

接着，我们需要提前按照格式准备好的数据集，并上传到智谱的服务器上。这里我们使用一个情感分类的数据集，您可以在同一个Json中包含多个文本并有多个不同的配置文件细节，例如，您可以指定某一条数据使用哪个模型，设置模型的参数和输出长度等。在本demo中，我已经为大家提供了一个数据集。这里的数据集均使用了`GLM-4`模型。

Next, we need to prepare the data set in advance according to the format and upload it to zhipuai's server. Here we use an emotion classification data set. You can include multiple texts in the same Json and have multiple different configuration file details. For example, you can specify which model to use for a certain piece of data, and set the parameters and output length of the model. In this demo, I have provided a data set for you. The data set here all use the `GLM-4` model.

In [6]:
result = client.files.create(
    file=open("data/batch_input.jsonl", "rb"),
    purpose="batch"
)
print(result.id)

1716468180_59c7ca593a0b4384bfda9d85cc64181d


# 3. Create a batch task

接着，我们需哟根据我们的上传任务，创建一个batch任务。在这个任务中，我们需要指定我们的输入文件，输出文件，以及我们的模型的endpoint。我们的endpoint是`/v4/chat/completions`。


Next, we need to create a batch task based on our upload task. In this task, we need to specify our input files, output files, and the endpoint of our model. Our endpoint is `/v4/chat/completions`.

In [8]:
create = client.batches.create(
    input_file_id="1716468180_59c7ca593a0b4384bfda9d85cc64181d",
    endpoint="/v4/chat/completions",
    completion_window="24h",
    metadata={
        "description": "Sentiment classification"
    }
)
print(create)

Batch(id='batch_1793623747699552256', completion_window='24h', created_at=1716468200026, endpoint='/v4/chat/completions', input_file_id='1716468180_59c7ca593a0b4384bfda9d85cc64181d', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=None, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Sentiment classification'}, output_file_id=None, request_counts=BatchRequestCounts(completed=None, failed=None, total=None))


# 4. Get task status
我们的任务已经创建，接下来我们需要获取任务的状态。我们可以通过任务的id来获取任务的状态。（在这个查询结果中，我等待了24小时才进行查询，确保任务已经完成并返回了结果)

Our task has been created, now we need to get the status of the task. We can get the status of the task through the task id. (In this query result, I waited 24 hours before querying to ensure that the task was completed and the results were returned)

In [4]:
batch_job = client.batches.retrieve(batch_id="batch_1793623747699552256")
print(batch_job)

Batch(id='batch_1793623747699552256', completion_window='24h', created_at=1716468200000, endpoint='/v4/chat/completions', input_file_id='1716468180_59c7ca593a0b4384bfda9d85cc64181d', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1716473500000, error_file_id='', errors=None, expired_at=None, expires_at=None, failed_at=None, finalizing_at=1716473460000, in_progress_at=1716468310000, metadata={'description': 'Sentiment classification'}, output_file_id='1716473500_874dd086fcc34f4fb56af30926ff959a', request_counts=BatchRequestCounts(completed=3, failed=0, total=3))


# 5. Get task status
在等待一天的时间后，我们可以通过任务的id来获取任务的状态，并将运行结果下载到本地。运行结束后，我们通过查看任务的状态，可以获取`output_file_id`。我们就可以获取解答的文件并下载。

After waiting for a day, we can get the status of the task through the task id and download the running results to the local. After running, we can get `output_file_id` by checking the status of the task. We can get the answer file and download it.

In [6]:
content = client.files.content(file_id="1716473500_874dd086fcc34f4fb56af30926ff959a")
content.write_to_file(file="data/batch_output.jsonl")

保存的文件`batch_output.jsonl` 已经同步上传到本案例中的`data`文件夹中。

The saved file `batch_output.jsonl` has been synchronously uploaded to the `data` folder in this case.